In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Code
!pip install transformers datasets accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 14.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 40.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━

**old code for whole dataset**

****new code for new dataset****

We are using 10% original training dataset for training the model and the other 10& of the training dataset for validation during training. The validation dataset in the original source is used for computing final f1 score

In [3]:
import transformers
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator

print(f"Transformers version: {transformers.__version__}")
print(f"Datasets version: {datasets.__version__}")


MODEL_CHECKPOINT = "distilbert-base-uncased"
BATCH_SIZE = 16
MAX_LENGTH = 384
DOC_STRIDE = 128

try:
    raw_datasets = load_dataset("lucadiliello/newsqa")
except Exception as e:
    print(f"Failed to load 'newsqa' dataset. Error: {e}")
    print("Please ensure you have internet access enabled in your Kaggle notebook.")

print("\n--- Original Datasets ---")
print(raw_datasets)


final_evaluation_dataset = raw_datasets["validation"]
print(f"\nSaved final evaluation set with {len(final_evaluation_dataset)} rows.")

shuffled_train = raw_datasets["train"].shuffle(seed=42)
train_size_10_percent = int(len(shuffled_train) * 0.1)
new_train_split = shuffled_train.select(range(train_size_10_percent))
new_valid_split = shuffled_train.select(range(train_size_10_percent, train_size_10_percent * 2))
raw_datasets["train"] = new_train_split
raw_datasets["validation"] = new_valid_split

print("\n--- Resampled Datasets for Training/Validation ---")
print(raw_datasets)
print("\n--- New Training Sample ---")
print(raw_datasets["train"][0])

Transformers version: 4.53.3
Datasets version: 4.1.1


README.md:   0%|          | 0.00/681 [00:00<?, ?B/s]

data/train-00000-of-00001-ec54fbe500fc3b(…):   0%|          | 0.00/29.7M [00:00<?, ?B/s]

data/validation-00000-of-00001-3cf888b12(…):   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/74160 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4212 [00:00<?, ? examples/s]


--- Original Datasets ---
DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', 'key', 'labels'],
        num_rows: 74160
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', 'key', 'labels'],
        num_rows: 4212
    })
})

Saved final evaluation set with 4212 rows.

--- Resampled Datasets for Training/Validation ---
DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers', 'key', 'labels'],
        num_rows: 7416
    })
    validation: Dataset({
        features: ['context', 'question', 'answers', 'key', 'labels'],
        num_rows: 7416
    })
})

--- New Training Sample ---
{'context': 'NEW DELHI, India (CNN) -- More than a dozen trucks loaded with fruits, spices and other goods Tuesday crossed the line dividing Kashmir as India and Pakistan opened a symbolic trade route in the disputed region for the first time in more than 60 years.\n\n\n\nKashmiris watch as Indian trucks cross int

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

if not tokenizer.is_fast:
    raise ValueError(
        "This script requires a fast tokenizer for 'return_offsets_mapping'."
        "Please use a different model checkpoint."
    )
    
pad_on_right = tokenizer.padding_side == "right"

def prepare_train_features(examples):
    """
    Preprocesses the training data.
    - Tokenizes question + context pairs.
    - Handles long contexts with striding.
    - Maps character answer spans to token spans.
    """
    examples["question"] = [q.strip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        labels = examples["labels"][sample_index]
    
        if not labels or not labels[0]['start']:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
            continue
        start_char = labels[0]['start'][0]
        end_char = labels[0]['end'][0]
        token_start_index = 0
        while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
            token_start_index += 1

        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
            token_end_index -= 1


        if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
           
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)

    
            while offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

tokenized_datasets = raw_datasets.map(
    prepare_train_features,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
    num_proc=2  
)

print("\n--- Tokenized Training Data Sample ---")
print(tokenized_datasets["train"][0])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map (num_proc=2):   0%|          | 0/7416 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/7416 [00:00<?, ? examples/s]


--- Tokenized Training Data Sample ---
{'input_ids': [101, 2054, 2003, 1037, 3120, 1997, 1037, 8618, 7593, 1029, 102, 2047, 6768, 1010, 2634, 1006, 13229, 1007, 1011, 1011, 2062, 2084, 1037, 6474, 9322, 8209, 2007, 10962, 1010, 21729, 1998, 2060, 5350, 9857, 4625, 1996, 2240, 16023, 13329, 2004, 2634, 1998, 4501, 2441, 1037, 12613, 3119, 2799, 1999, 1996, 11621, 2555, 2005, 1996, 2034, 2051, 1999, 2062, 2084, 3438, 2086, 1012, 13329, 2483, 3422, 2004, 2796, 9322, 2892, 2046, 1996, 3675, 2237, 1997, 15775, 23342, 4048, 1999, 9889, 1011, 8564, 13329, 1012, 2000, 21250, 1998, 2316, 2189, 1010, 1996, 2034, 9322, 22257, 2013, 1996, 9889, 2217, 2408, 1037, 2317, 2958, 2000, 1037, 14224, 7429, 3119, 2415, 17037, 7828, 2098, 1012, 6207, 1011, 14887, 2796, 9322, 5825, 1996, 2060, 2126, 2020, 2036, 22775, 10979, 1012, 2009, 2003, 5113, 1996, 2693, 2097, 7496, 13136, 1999, 1996, 11587, 28333, 2555, 1010, 2029, 2038, 2042, 1037, 4736, 5956, 8400, 2090, 4517, 1011, 4273, 10638, 2634, 1998, 4501, 2

In [5]:

from transformers import EarlyStoppingCallback

model = AutoModelForQuestionAnswering.from_pretrained(MODEL_CHECKPOINT)

args = TrainingArguments(
    output_dir="finetuned-newsqa",
    eval_strategy="epoch",
    save_strategy="epoch", 
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_37/1954319160.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
from transformers import EarlyStoppingCallback

model = AutoModelForQuestionAnswering.from_pretrained(MODEL_CHECKPOINT)

args = TrainingArguments(
    output_dir="finetuned-newsqa",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=10, 
    weight_decay=0.01,
    fp16=True,
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="loss",  
    greater_is_better=False,      
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_37/3715813353.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:

print("--- Starting Training ---")
train_result = trainer.train()
print("--- Training Finished ---")

trainer.save_model()  
train_metrics = train_result.metrics
trainer.log_metrics("train", train_metrics)
trainer.save_metrics("train", train_metrics)
trainer.save_state()

--- Starting Training ---


Epoch,Training Loss,Validation Loss
1,1.856200,1.644364
2,1.308300,1.356617
3,0.959800,1.406179
4,0.660500,1.507911


--- Training Finished ---
***** train metrics *****
  epoch                    =        4.0
  total_flos               =  6801070GF
  train_loss               =     1.2659
  train_runtime            = 0:30:54.12
  train_samples_per_second =    100.484
  train_steps_per_second   =      6.283


In [11]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [15]:
import torch
import numpy as np
import collections
from tqdm.auto import tqdm
import evaluate 

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size=20, max_answer_length=30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["key"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["key"]]].append(i)

    predictions = collections.OrderedDict()
    print(f"Post-processing {len(examples)} examples split into {len(features)} features.")

    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]
        min_null_score = None
        valid_answers = []
        context = example["context"]
        
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or end_index < start_index
                        or start_index == cls_index
                        or end_index == cls_index
                        or (end_index - start_index + 1) > max_answer_length
                    ):
                        continue
                    
                
                    if not (features[feature_index]["sequence_ids"][start_index] == (1 if pad_on_right else 0) and
                            features[feature_index]["sequence_ids"][end_index] == (1 if pad_on_right else 0)):
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["key"]] = best_answer["text"]

    return predictions



print("\n--- Running Predictions on FINAL VALIDATION SET ---")

-
def prepare_validation_features(examples):
    examples["question"] = [q.strip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    tokenized_examples["key"] = []
    tokenized_examples["sequence_ids"] = [] 
    
    for i in range(len(tokenized_examples["input_ids"])):
        sample_index = sample_mapping[i]
        tokenized_examples["key"].append(examples["key"][sample_index])

        # Get the sequence ids
        sequence_ids = tokenized_examples.sequence_ids(i)
        tokenized_examples["sequence_ids"].append(sequence_ids)
        
        # Set offset_mapping to None for question tokens
        offset_mapping = tokenized_examples["offset_mapping"][i]
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == (1 if pad_on_right else 0) else None)
            for k, o in enumerate(offset_mapping)
        ]
    return tokenized_examples

validation_features = final_evaluation_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=final_evaluation_dataset.column_names,
    num_proc=2
)

final_raw_predictions = trainer.predict(validation_features)

final_predictions = postprocess_qa_predictions(
    final_evaluation_dataset,
    validation_features,
    final_raw_predictions.predictions
)


metric = evaluate.load("squad")

formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]

formatted_references = []
for ex in final_evaluation_dataset:
    formatted_references.append({
        "id": ex["key"],
        "answers": {
            "text": ex["answers"],
            "answer_start": [l['start'][0] for l in ex['labels'] if l['start']]
        }
    })

results = metric.compute(predictions=formatted_predictions, references=formatted_references)

print("\n--- FINAL Evaluation Results (on full validation set) ---")
print(f"Exact Match (EM): {results['exact_match']:.2f}%")
print(f"F1 Score: {results['f1']:.2f}%")


--- Running Predictions on FINAL VALIDATION SET ---


Post-processing 4212 examples split into 10567 features.


  0%|          | 0/4212 [00:00<?, ?it/s]


--- FINAL Evaluation Results (on full validation set) ---
Exact Match (EM): 34.40%
F1 Score: 48.17%
